# ACRO Tests

In [1]:
import os
import sys
import pandas as pd
import numpy as np

In [2]:
sys.path.insert(0, os.path.abspath(".."))

In [3]:
from acro import ACRO, add_constant

### Instantiate ACRO

In [4]:
acro = ACRO()

INFO:acro:config: {'safe_threshold': 10, 'safe_dof_threshold': 10, 'safe_nk_n': 2, 'safe_nk_k': 0.9, 'safe_pratio_p': 0.1, 'check_missing_values': True}


### Load test data

In [5]:
path = os.path.join("../data", "test_data.dta")
df = pd.read_stata(path)
df.head()

,charity,grant_type,index,year,inc_activity,inc_grants,inc_donations,inc_other,inc_total,total_costs,...,sh_staff_grants_given,sh_assets_grants_given,sh_income_balance,sh_staff_balance,sh_assets_balance,sh_income_assets,sh_staff_assets,sh_income_staff_costs,sh_assets_staff_costs,wgt
0,4Children,R,1.0,2011,2880902.0,9603182.0,91404.0,310947.0,12886435.0,12127472.0,...,NaN,NaN,0.072636,0.135971,0.767809,0.094602,0.177090,0.534203,5.646843,1.0
1,4Children,R,1.0,2014,6810520.0,18768904.0,58002.0,401879.0,26039304.0,25493796.0,...,NaN,NaN,0.057641,0.089150,1.001396,0.057560,0.089026,0.646561,11.232729,1.0
2,4Children,R,1.0,2015,7199403.0,21638036.0,132191.0,512654.0,29482284.0,32290108.0,...,NaN,NaN,-0.049619,-0.079828,-0.620210,0.080004,0.128711,0.621583,7.769365,1.0
3,4Children,R,1.0,2013,5573013.0,15194731.0,228844.0,267156.0,21263744.0,20989048.0,...,NaN,NaN,0.045740,0.068251,1.008259,0.045365,0.067692,0.670166,14.772749,1.0
4,4Children,R,1.0,2010,2056816.0,7335103.0,110256.0,424628.0,9926803.0,9769816.0,...,NaN,NaN,0.057696,0.122532,0.567539,0.101660,0.215901,0.470862,4.631749,1.0


### Pandas crosstab

In [6]:
table = pd.crosstab(df.year, df.grant_type)
table

grant_type,G,N,R,R/G
year,,,,
2010,15,59,71,8
2011,15,59,71,8
2012,15,59,71,8
2013,15,59,71,8
2014,15,59,71,8
2015,15,59,71,8


### ACRO crosstab

In [7]:
safe_table = acro.crosstab(df.year, df.grant_type)
safe_table

INFO:acro:outcome_df:
grant_type   G   N   R          R/G
year                               
2010        ok  ok  ok  threshold; 
2011        ok  ok  ok  threshold; 
2012        ok  ok  ok  threshold; 
2013        ok  ok  ok  threshold; 
2014        ok  ok  ok  threshold; 
2015        ok  ok  ok  threshold; 
INFO:acro:get_summary(): fail; threshold: 6 cells suppressed; 
INFO:acro:add_output(): output_0_2023-04-17-17211066


grant_type,G,N,R,R/G
year,,,,
2010,15,59,71,NaN
2011,15,59,71,NaN
2012,15,59,71,NaN
2013,15,59,71,NaN
2014,15,59,71,NaN
2015,15,59,71,NaN


### ACRO crosstab with aggregation function and ACRO crosstab with missing values (if the default check_missing_values in the yaml file is set to True)

In [8]:
safe_table = acro.crosstab(df.year, df.grant_type, values=df.inc_grants, aggfunc="mean")
safe_table

INFO:acro:outcome_df:
grant_type          G          N          R  \
year                                          
2010        missing;   missing;   missing;    
2011               ok  missing;   missing;    
2012               ok         ok  missing;    
2013               ok         ok         ok   
2014               ok         ok  missing;    
2015        missing;   missing;   missing;    

grant_type                                     R/G  
year                                                
2010        threshold; p-ratio; nk-rule; missing;   
2011                                   threshold;   
2012                                   threshold;   
2013                                   threshold;   
2014                                   threshold;   
2015                                   threshold;   
INFO:acro:get_summary(): review; missing values found
INFO:acro:add_output(): output_1_2023-04-17-17211081


grant_type,G,N,R,R/G
year,,,,
2010,NaN,NaN,NaN,NaN
2011,8502247.0,NaN,NaN,NaN
2012,11458580.0,131859.062500,NaN,NaN
2013,13557147.0,147937.796875,7202273.5,NaN
2014,13748147.0,133198.250000,NaN,NaN
2015,NaN,NaN,NaN,NaN


### ACRO crosstab with negative values

In [9]:
negative = df.inc_grants.copy()
negative[0:10] = -10

safe_table = acro.crosstab(df.year, df.grant_type, values=negative, aggfunc="mean")
safe_table

INFO:acro:outcome_df:
grant_type G         N         R R/G
year                                
2010                    negative    
2011          negative  negative    
2012                    negative    
2013          negative  negative    
2014          negative  negative    
2015          negative  negative    
INFO:acro:get_summary(): review; negative values found
INFO:acro:add_output(): output_2_2023-04-17-17211094


grant_type,G,N,R,R/G
year,,,,
2010,9921906.0,0.000000,8280032.5,11636000.0
2011,8502247.0,123496.445312,7577703.5,16047500.0
2012,11458580.0,131859.062500,6796357.5,16810000.0
2013,13557147.0,147937.625000,6988263.5,16765625.0
2014,13748147.0,133198.078125,7997392.5,17845750.0
2015,11133433.0,146572.015625,10388613.0,18278624.0


### ACRO pivot_table

In [10]:
table = acro.pivot_table(
    df, index=["grant_type"], values=["inc_grants"], aggfunc=["mean", "std"]
)
table

TypeError: 'bool' object is not callable

### ACRO pivot_table with negative values

In [ ]:
df.loc[0:10, "inc_grants"] = -10

table = acro.pivot_table(
    df, index=["grant_type"], values=["inc_grants"], aggfunc=["mean", "std"]
)
table

INFO:acro:outcome_df:
                 mean        std
           inc_grants inc_grants
grant_type                      
G                               
N            negative   negative
R            negative   negative
R/G                             
INFO:acro:get_summary(): review; negative values found
INFO:acro:add_output(): output_4_2023-04-17-17175704


,mean,std
,inc_grants,inc_grants
grant_type,,
G,1.141279e+07,2.283220e+07
N,1.341800e+05,1.990196e+05
R,7.882231e+06,3.204558e+07
R/G,1.664827e+07,1.583532e+07


### ACRO OLS

In [ ]:
new_df = df[["inc_activity", "inc_grants", "inc_donations", "total_costs"]]
new_df = new_df.dropna()

y = new_df["inc_activity"]
x = new_df[["inc_grants", "inc_donations", "total_costs"]]
x = add_constant(x)

results = acro.ols(y, x)
results.summary()

INFO:acro:ols() outcome: pass; dof=807.0 >= 10
INFO:acro:add_output(): output_5_2023-04-17-17175717


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           inc_activity   R-squared:                       0.894
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     2261.
Date:                Mon, 17 Apr 2023   Prob (F-statistic):               0.00
Time:                        17:17:57   Log-Likelihood:                -14495.
No. Observations:                 811   AIC:                         2.900e+04
Df Residuals:                     807   BIC:                         2.902e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           3.01e+05   5.33e+05      0.565      0.572   -7.45e+05    1.35e+06
inc_grants       -0.8846      0.025    -35.956      0.000      -0.933      -0.836
inc_donations    -0.6647      0.016    -40.721      0.000      -0.697      -0.633
total_costs       0.8313      0.011     78.674      0.000       0.811       0.852
==============================================================================
Omnibus:                     1339.956   Durbin-Watson:                   1.414
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1253317.706
Skew:                           9.899   Prob(JB):                         0.00
Kurtosis:                     194.566   Cond. No.                     1.05e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### ACRO OLSR

In [ ]:
results = acro.olsr(
    formula="inc_activity ~ inc_grants + inc_donations + total_costs", data=new_df
)
results.summary()

INFO:acro:olsr() outcome: pass; dof=807.0 >= 10
INFO:acro:add_output(): output_6_2023-04-17-17175727


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           inc_activity   R-squared:                       0.894
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     2261.
Date:                Mon, 17 Apr 2023   Prob (F-statistic):               0.00
Time:                        17:17:57   Log-Likelihood:                -14495.
No. Observations:                 811   AIC:                         2.900e+04
Df Residuals:                     807   BIC:                         2.902e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       3.01e+05   5.33e+05      0.565      0.572   -7.45e+05    1.35e+06
inc_grants       -0.8846      0.025    -35.956      0.000      -0.933      -0.836
inc_donations    -0.6647      0.016    -40.721      0.000      -0.697      -0.633
total_costs       0.8313      0.011     78.674      0.000       0.811       0.852
==============================================================================
Omnibus:                     1339.956   Durbin-Watson:                   1.414
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1253317.706
Skew:                           9.899   Prob(JB):                         0.00
Kurtosis:                     194.566   Cond. No.                     1.05e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### ACRO Probit

In [ ]:
new_df = df[["survivor", "inc_activity", "inc_grants", "inc_donations", "total_costs"]]
new_df = new_df.dropna()

y = new_df["survivor"].astype("category").cat.codes  # numeric
y.name = "survivor"
x = new_df[["inc_activity", "inc_grants", "inc_donations", "total_costs"]]
x = add_constant(x)

results = acro.probit(y, x)
results.summary()

INFO:acro:probit() outcome: pass; dof=806.0 >= 10
INFO:acro:add_output(): output_7_2023-04-17-17175741


Optimization terminated successfully.
         Current function value: 0.493791
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:               survivor   No. Observations:                  811
Model:                         Probit   Df Residuals:                      806
Method:                           MLE   Df Model:                            4
Date:                Mon, 17 Apr 2023   Pseudo R-squ.:                  0.2140
Time:                        17:17:57   Log-Likelihood:                -400.46
converged:                       True   LL-Null:                       -509.50
Covariance Type:            nonrobust   LLR p-value:                 4.875e-46
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0474      0.057      0.838      0.402      -0.063       0.158
inc_activity   1.836e-07   5.16e-08      3.559      0.000    8.25e-08    2.85e-07
inc_grants     8.576e-08    3.9e-08      2.197      0.028    9.25e-09    1.62e-07
inc_donations  2.406e-07   4.54e-08      5.297      0.000    1.52e-07     3.3e-07
total_costs   -8.644e-08   3.68e-08     -2.351      0.019   -1.59e-07   -1.44e-08
=================================================================================

Possibly complete quasi-separation: A fraction 0.18 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

### ACRO Logit

In [ ]:
results = acro.logit(y, x)
results.summary()

INFO:acro:logit() outcome: pass; dof=806.0 >= 10
INFO:acro:add_output(): output_8_2023-04-17-17175753


Optimization terminated successfully.
         Current function value: 0.490836
         Iterations 12


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               survivor   No. Observations:                  811
Model:                          Logit   Df Residuals:                      806
Method:                           MLE   Df Model:                            4
Date:                Mon, 17 Apr 2023   Pseudo R-squ.:                  0.2187
Time:                        17:17:57   Log-Likelihood:                -398.07
converged:                       True   LL-Null:                       -509.50
Covariance Type:            nonrobust   LLR p-value:                 4.532e-47
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0512      0.091      0.561      0.575      -0.128       0.230
inc_activity   2.981e-07   8.95e-08      3.330      0.001    1.23e-07    4.74e-07
inc_grants     1.351e-07   6.67e-08      2.026      0.043    4.39e-09    2.66e-07
inc_donations  5.123e-07   1.04e-07      4.927      0.000    3.08e-07    7.16e-07
total_costs   -1.442e-07   6.26e-08     -2.304      0.021   -2.67e-07   -2.15e-08
=================================================================================

Possibly complete quasi-separation: A fraction 0.18 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

### List current ACRO outputs

In [ ]:
acro.print_outputs()

output_0_2023-04-17-17175635:
command: safe_table = acro.crosstab(df.year, df.grant_type)
summary: fail; threshold: 6 cells suppressed; 
outcome: grant_type   G   N   R          R/G
year                               
2010        ok  ok  ok  threshold; 
2011        ok  ok  ok  threshold; 
2012        ok  ok  ok  threshold; 
2013        ok  ok  ok  threshold; 
2014        ok  ok  ok  threshold; 
2015        ok  ok  ok  threshold; 
output: [grant_type   G   N   R  R/G
year                       
2010        15  59  71  NaN
2011        15  59  71  NaN
2012        15  59  71  NaN
2013        15  59  71  NaN
2014        15  59  71  NaN
2015        15  59  71  NaN]
timestamp: 2023-04-17-17175635
comments: 


output_1_2023-04-17-17175652:
command: safe_table = acro.crosstab(df.year, df.grant_type, values=df.inc_grants, aggfunc="mean")
summary: fail; threshold: 6 cells suppressed; p-ratio: 1 cells suppressed; nk-rule: 1 cells suppressed; 
outcome: grant_type   G   N   R                        

### Remove some ACRO outputs before finalising

In [ ]:
output_1 = list(acro.results.keys())[1]
output_4 = list(acro.results.keys())[4]

acro.remove_output(output_1)
acro.remove_output(output_4)

INFO:acro:remove_output(): output_1_2023-04-17-17175652 removed
INFO:acro:remove_output(): output_4_2023-04-17-17175704 removed


### Rename ACRO outputs before finalising

In [ ]:
acro.rename_output(list(acro.results.keys())[2], "pivot_table")

INFO:acro:rename_output(): output_3_2023-04-17-17175684 renamed to pivot_table_2023-04-17-17175684


### Add a comment to output

In [ ]:
acro.add_comments(
    list(acro.results.keys())[0], "This is a cross table between year and grant_type"
)
acro.add_comments(list(acro.results.keys())[0], "6 cells were supressed in this table")

INFO:acro:a comment was added to output_0_2023-04-17-17175635
INFO:acro:a comment was added to output_0_2023-04-17-17175635


### Add an unsupported output to the list of outputs

In [ ]:
acro.custom_output(
    "XandY.jfif", "This output is an image showing the relationship between X and Y"
)

INFO:acro:add_output(): output_9_2023-04-17-17175797


### Finalise ACRO

In [ ]:
# output = acro.finalise("test_results.xlsx")
output = acro.finalise("test_results.json")

INFO:acro:Directory outputs/ already exists
INFO:acro:output written to: test_results.json
